In [5]:
import dowhy
import networkx as nx
import pandas as pd
import numpy as np

In [125]:
n_samples = 1000
random_sample = lambda: np.random.normal(loc = 10, size=n_samples) 
random_binary_sample = lambda p=0.5: np.random.choice([0, 1], size=n_samples, p=(1-p, p)).astype(bool)

biased_random_binary_sample = lambda cond, p1, p2: np.where(cond, random_binary_sample(p1), random_binary_sample(p2))

_ = random_binary_sample()
_2 = biased_random_binary_sample(_, 0.3, 0.8)

df = pd.DataFrame({
    "x_age": random_sample(),
    "x_sex": _,
    "d_d0": random_binary_sample(),
    "d_d1": _2,
    "y_K": random_sample(),
    "y_A": random_sample() + 2 * _2 - 1.67 * _
})

In [195]:
def get_diagnoses(df):
    return [d for d in df.columns if d.startswith("d_")]

def get_other_covariates(df):
    return [x for x in df.columns if x.startswith("x_")]

def get_outcome(df):
    return [y for y in df.columns if y.startswith("y_")]


def construct_network_indep_diagnosis(df):
    x_to_y = [(x, y) for x in get_other_covariates(df) for y in get_outcome(df)]
    x_to_d = [(x, d) for x in get_other_covariates(df) for d in get_diagnoses(df)]
    d_to_y = [(d, y) for d in get_diagnoses(df) for y in get_outcome(df)]
    
    return str(nx.nx_pydot.to_pydot(nx.DiGraph(x_to_y + x_to_d + d_to_y)))


def construct_network_full(df):
    all_covariates = get_other_covariates(df) + get_diagnoses(df)
    return str(nx.nx_pydot.to_pydot(nx.DiGraph(
        [(x, y) for x in all_covariates for y in all_covariates] + [(x, y) for x in all_covariates for y in get_outcome(df)]
    )))



def construct_network_indep_diagnosis_to_risk(df):
    x_to_y = [(x, y) for x in get_other_covariates(df) for y in get_outcome(df)]
    x_to_d = [(x, d) for x in get_other_covariates(df) for d in get_diagnoses(df)]
    d_to_y = [(d, y) for d in get_diagnoses(df) for y in get_outcome(df)]
    y_to_risk = [(y, "Risk") for y in get_outcome(df)]
    
    return str(nx.nx_pydot.to_pydot(nx.DiGraph(x_to_y + x_to_d + d_to_y + y_to_risk)))


# Playground

In [277]:
# m = dowhy.CausalModel(df, treatment=get_diagnoses(df), outcome=get_outcome(df), graph=construct_network_indep_diagnosis(df))
# m = dowhy.CausalModel(df, treatment=["d_d0", "d_d1"], outcome="y_A", graph=construct_network_indep_diagnosis(df))
# m = dowhy.CausalModel(df, treatment=["d_d0", "d_d1"], outcome="y_A", graph=construct_network_indep_diagnosis(df))
m = dowhy.CausalModel(df, treatment="d_d1", outcome="y_A", graph=construct_network_indep_diagnosis(df))
# m = dowhy.CausalModel(df, treatment=["d_d0", "d_d1"], outcome="y_A", graph=construct_network_full(df))

In [278]:
identified = m.identify_effect(proceed_when_unidentifiable=True)
# print(identified)

estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True, confidence_intervals=True)
# estimate = m.estimate_effect(identified, method_name="backdoor.distance_matching") #, test_significance=True)

print("\n\n===")
print(estimate)

print("\n\n===")
# ref = m.refute_estimate(identified, estimate, method_name="random_common_cause")
# ref = m.refute_estimate(identified, estimate, method_name="add_unobserved_common_cause")
# print(ref.refutation_result)

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': True, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_value': 0, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_d0']}
{'treatment_value': 1, 'control_va

In [188]:
# estimate.get_confidence_intervals()
# estimate.test_stat_significance()
# estimate.params
# estimate.estimate_conditional_effects("d_d1")
estimate.value

# Data Analysis

In [238]:
def causal_estimate_independent_diagnoses(df, graph, outcome):
    ds = get_diagnoses(df)[:5]
    
    effects = {}
    
    for diagnosis in ds:
        m = dowhy.CausalModel(df, treatment=diagnosis, outcome=outcome, graph=graph)
        
        identified = m.identify_effect(proceed_when_unidentifiable=True)
        # print(identified)

        estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True, confidence_intervals=True)
        # estimate = m.estimate_effect(identified, method_name="backdoor.distance_matching") #, test_significance=True)

        # print("===")
        # print(estimate)
        
        effects[diagnosis] = (estimate, estimate.value)
    
    return effects

In [ ]:
def causal_estimate_independent_diagnoses(df, graph, outcome):
    ds = get_diagnoses(df)
    
    effects = {}
    
    for diagnosis in ds:
        m = dowhy.CausalModel(df, treatment=diagnosis, outcome=outcome, graph=graph)
        
        identified = m.identify_effect(proceed_when_unidentifiable=True)
        print(identified)

        estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True)
        # estimate = m.estimate_effect(identified, method_name="backdoor.distance_matching") #, test_significance=True)

        # print("===")
        # print(estimate)
        
        effects[diagnosis].append((estimate, estimate.value))
    
    return effects

In [275]:
real_df


ckd = "N18.3"
hyper = "I10"

In [274]:
real_df = pd.read_csv("../data/honza_data.csv")
# print(get_diagnoses(real_df)[:10] + get_outcome(real_df) + get_other_covariates(real_df))
# i = 
# real_df = real_df[get_diagnoses(real_df)[i:i+1] + get_outcome(real_df) + get_other_covariates(real_df)]
real_df = real_df.loc[~real_df.y_ACR.isna()]
# real_df.head()

In [ ]:
# graph = construct_network_indep_diagnosis(real_df)

In [276]:
m = dowhy.CausalModel(real_df, treatment=get_diagnoses(real_df), outcome="y_ACR", graph=construct_network_indep_diagnosis(real_df))
        
identified = m.identify_effect(proceed_when_unidentifiable=True)
print(identified)

estimate = m.estimate_effect(identified, method_name="backdoor.linear_regression", test_significance=True, confidence_intervals=True)
print("\n\n\n===========")
print(estimate)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
                                                                                                                                                                                                                                          
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
d[d_J30.2  d_J30.3  d_J45.8  d_I10  d_I25.9  d_Z95.1  d_U58.9  d_E78.2  d_E11.9  d_H40.9  d_I48.0  d_J93.9  d_I50.9  d_Z95.3  d_I48.1  d_I36.1  d_I27.2  d_Z95.0  d_I34.0  d_E78.5  d_G20  d_I46.0  d_I44.2  d_N95.1  d_I21.1  d_I50.1  d_

                                                                                                                                                                                                      

In [264]:
a = estimate.get_confidence_intervals()
a[(a > 0.2).all(axis=1)]
# (a > 0.2).sum(axis=1)

array([], shape=(0, 2), dtype=float64)

In [229]:
causal_estimate_independent_diagnoses(real_df, graph, "y_ACR")

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_I25.9', 'd_J30.3', 'd_E11.9', 'd_J45.8', 'd_H40.9', 'd_U58.9', 'd_E78.2', 'd_Z95.1', 'd_I10']}


/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/dowhy/causal_estimator.py:260: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  conditional_estimates = by_effect_mods.apply(estimate_effect_fn)


linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_I25.9', 'd_E11.9', 'd_J45.8', 'd_H40.9', 'd_U58.9', 'd_E78.2', 'd_J30.2', 'd_Z95.1', 'd_I10']}


/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/dowhy/causal_estimator.py:260: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  conditional_estimates = by_effect_mods.apply(estimate_effect_fn)


linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_I25.9', 'd_J30.3', 'd_E11.9', 'd_H40.9', 'd_U58.9', 'd_E78.2', 'd_J30.2', 'd_Z95.1', 'd_I10']}


...

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_I25.9', 'd_J30.3', 'd_E11.9', 'd_J45.8', 'd_H40.9', 'd_U58.9', 'd_E78.2', 'd_J30.2', 'd_Z95.1']}


/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/dowhy/causal_estimator.py:260: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  conditional_estimates = by_effect_mods.apply(estimate_effect_fn)


linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': True, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['d_E11.9', 'd_J30.3', 'd_J45.8', 'd_H40.9', 'd_U58.9', 'd_E78.2', 'd_J30.2', 'd_Z95.1', 'd_I10']}


/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/home/jan/mambaforge/envs/hhh/lib/python3.10/site-packages/dowhy/causal_estimator.py:260: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  conditional_estimates = by_effect_mods.apply(estimate_effect_fn)


{'d_J30.2': (<dowhy.causal_estimator.CausalEstimate at 0x7feaf8d55240>,
  1.8474111129762605e-13),
 'd_J30.3': (<dowhy.causal_estimator.CausalEstimate at 0x7feaeb75fbe0>,
  1.8474111129762605e-13),
 'd_J45.8': (<dowhy.causal_estimator.CausalEstimate at 0x7feaeb3a3a90>,
  1.8474111129762605e-13),
 'd_I10': (<dowhy.causal_estimator.CausalEstimate at 0x7feaeaf4eb60>,
  1.8474111129762605e-13),
 'd_I25.9': (<dowhy.causal_estimator.CausalEstimate at 0x7feaea890c70>,
  1.8474111129762605e-13)}

In [246]:
real_df.head()
real_df.describe()

,y_KREA,y_ACR,x_age,x_BMI
count,10377.000000,10382.000000,10382.000000,10382.000000
mean,119.624313,12.525905,59.618821,28.406136
std,96.253719,41.425530,16.240780,5.974171
min,28.100000,0.000000,16.300000,12.600000
25%,75.000000,0.700000,48.100000,24.500000
50%,93.100000,1.600000,61.500000,27.800000
75%,127.500000,5.900000,72.300000,31.600000
max,1363.400000,865.800000,100.200000,227.200000


In [243]:
real_df.describe()

,y_KREA,y_ACR,x_age,x_BMI
count,10377.000000,10382.000000,10382.000000,10382.000000
mean,119.624313,12.525905,59.618821,28.406136
std,96.253719,41.425530,16.240780,5.974171
min,28.100000,0.000000,16.300000,12.600000
25%,75.000000,0.700000,48.100000,24.500000
50%,93.100000,1.600000,61.500000,27.800000
75%,127.500000,5.900000,72.300000,31.600000
max,1363.400000,865.800000,100.200000,227.200000
